In [ ]:
# import packages
import pandas as pd
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib_venn as venn
from helper import *
import ipywidgets as widgets

# variables
time_period = (1, 2, 3, 4, 5) # 5 different implementation of progression order  
n_patients = 0 # number of patients included in the model  
n_sessions = 0 # number of sessions included in the model  
usage_time: how many months the patient has been using the app  
usage_freq: how frequent the patient uses the app  

Clean Data

In [ ]:
# querying from the SQL database, 2 mins for q3 table

from connection import *

con = connect()
df = SQL("select * from constant_therapy.q3", con)
df = df.rename(columns={'session_id':"id"})

In [ ]:
# 1 min, data saved from previous runs
df = pd.read_csv("data/context_action.csv")
df.drop(df.columns[[0, 1]], axis=1, inplace=True)
df.head()

In [ ]:
# 20 seconds
disorder_ids = df.groupby("patient_id")["disorder_id"].apply(set).reset_index()
df = df.drop(columns="disorder_id")
df = df.merge(disorder_ids, on="patient_id", how="left")

In [ ]:
# 1 min
domain_ids = df.groupby("id")["domain_id"].apply(set).reset_index()
df = df.drop(columns="domain_id")
df = df.merge(domain_ids, on="id", how="left")

In [ ]:
df = df.drop_duplicates(subset="id")

In [ ]:
# df.to_csv("data/consolidate_data.csv")
## careful when reading, will read set as string
# df = pd.read_csv("data/consolidate_data.csv", index_col=[0])
df

All data has been consolidated, there should be no duplicate sessions for different disorders/domains.
We want to add time_period, usage_time, and usage_freq to the dataset.

**This part doesn't need to be run if it's claire's data**

In [ ]:
progression_order_df = pd.read_csv("data/progression_order.csv", index_col=[0])
progression_order_df.rename(columns={'time_implemented':'end_time'}, inplace=True)
progression_order_df["end_time"] = pd.to_datetime(progression_order_df["end_time"])
df["end_time"] = pd.to_datetime(df["end_time"])

In [ ]:
def time_period_convert(dd):
    dates = sorted(pd.unique(progression_order_df["end_time"]))
    if dd < pd.to_datetime(dates[1]):
        return 1
    elif dd < pd.to_datetime(dates[2]):
        return 2
    elif dd < pd.to_datetime(dates[3]):
        return 3
    elif dd < pd.to_datetime(dates[4]):
        return 4
    else:
        return 5

In [ ]:
# added time period, takes around 8-9 minutes to run
df["time_period"] = df["end_time"].apply(time_period_convert)
df

In [ ]:
progression_order_df["time_period"] = progression_order_df["end_time"].apply(time_period_convert)
progression_order_df

Adding Usage Time
* this is by patient
* need a patient dataframe vs session dataframe (df)

In [ ]:
df["end_time"] = pd.to_datetime(df["end_time"])

In [ ]:
patients = (df.groupby("patient_id")["end_time"].max() - df.groupby("patient_id")["end_time"].min()).reset_index()
patients.columns = ["patient_id", "usage_time"]
patients["usage_time"] = patients["usage_time"].dt.days + 1
patients

Add usage_freq for patient dataframe
* using the simplest way of calculating frequency
* how to get rid of outlier

In [ ]:
patients["session_count"] = df.groupby("patient_id")["id"].count().reset_index(name="session_count")["session_count"]
patients["unique_days"] = df.groupby("patient_id")["end_time"].nunique().reset_index(name="days")["days"]
patients

In [ ]:
patients["usage_freq"] = patients["unique_days"] / patients["usage_time"]
patients

How to remove outlier

In [ ]:
# modified from https://stackoverflow.com/questions/56750841/how-to-trim-outliers-in-dates-in-python
def datetime_outlier(data):
    qa = data["end_time"].quantile(0.1) #lower 10%
    qb = data["end_time"] #higher 10%
    #remove outliers
    xf = data[(data.end_time >= qa) & (data.end_time <= qb)]
    return xf

In [ ]:
# about a min
patients_v2 = df.groupby("patient_id").apply(datetime_outlier).reset_index(drop=True)

In [ ]:
patients_v2

In [ ]:
patients_filtered = (patients_v2.groupby("patient_id")["end_time"].max() - patients_v2.groupby("patient_id")["end_time"].min()).reset_index()
patients_filtered.columns = ["patient_id", "usage_time"]
patients_filtered["usage_time"] = patients_filtered["usage_time"].dt.days + 1
patients_filtered

In [ ]:
patients_filtered["session_count"] = patients_v2.groupby("patient_id")["id"].count().reset_index(name="session_count")["session_count"]
patients_filtered["unique_days"] = patients_v2.groupby("patient_id")["end_time"].nunique().reset_index(name="days")["days"]
patients_filtered["usage_freq"] = patients_filtered["unique_days"] / patients_filtered["usage_time"]

Check that the length of patients_filtered is less than the length of patients

In [ ]:
patients

In [ ]:
patients_filtered

## INTERACT
Filter seems to work for time outlier, so we using patient_filtered

In [ ]:
# interact to figure out what thresholds to use

@widgets.interact_manual(usage_time=(1, 365), usage_freq=(0.0, 1.0))
def visualize(usage_time, usage_freq):
    temp = patients_filtered[patients_filtered.usage_time > usage_time]
    temp = temp[temp.usage_freq > usage_freq]
    return "number of patients: %d" %(temp["patient_id"].nunique())

In [ ]:
# create the according filtered patients dataframe

usage_time = input("usage time: ")
print("inputted ", usage_time)
usage_freq = input("usage_freq: ")
print("inputted ", usage_freq)

In [ ]:
filtered_patients_list = patients_filtered[patients_filtered.usage_time > float(usage_time)]
filtered_patients_list = filtered_patients_list[filtered_patients_list.usage_freq > float(usage_freq)]

In [ ]:
filtered_patients_list.nunique()

In [ ]:
# create filtered session dataframe from filtered patients data
temp_lst = filtered_patients_list["patient_id"]
sessions_filter_df = df[df.patient_id.isin(temp_lst)]
sessions_filter_df

In [ ]:
# filter based on time_period
time_period_n = input("time period: ")
sessions_filter_df = sessions_filter_df[sessions_filter_df.time_period == int(time_period_n)]

In [ ]:
# sessions_filter_df.to_csv("data/PLACEHOLDER.csv")

In [ ]:
sessions_filter_df

Now that we have the filtered dataframe, we want to start performance metric calculation
* add progression order  
**for now**  
* calculate percentile of each session for each domain
* for each patient, average percentile value across all domains **at the time**, which is our final performance metric calculation

This part doesn not need to be run if it's claire's data

In [ ]:
# expand sessions due to compressed domain
sessions_filter_df.explode("domain_id").reset_index()

In [ ]:
progression_order_df[progression_order_df.time_period == int(time_period_n)]

In [ ]:
sessions_filter_df.dtypes

In [ ]:
def set_to_int(s):
    return s.pop()

In [ ]:
sessions_filter_df["domain_id"] = sessions_filter_df["domain_id"].apply(set_to_int)
sessions_filter_df

In [ ]:
# add progression order
sessions_filter_df = sessions_filter_df.merge(progression_order_df[progression_order_df.time_period == int(time_period_n)], on=["task_type_id", "task_level", "domain_id"]).reset_index()
sessions_filter_df

Get Percentile

In [ ]:
# get percentile of a domain
def get_percentile(data):
    data["percentile"] = data["progression_order"].rank(pct=True)
    return data

In [ ]:
## get percentile for each domain
df_pct = sessions_filter_df.groupby("domain_id").apply(get_percentile).reset_index(drop=True)
df_pct

In [ ]:
## clean dataframe, does not need to be run if Claire's data
df_pct.drop(df_pct.columns[[0, 11, 16, 17, 19, 20]], axis=1, inplace=True)
df_pct

In [ ]:
#df_pct.to_csv("data/raw_percentile.csv", index=False)

Get overall

In [ ]:
# take in individual dataframe and output dataframe with overall performance metric
def get_score(data):
    updated_domain_pct = dict() # keeps updated domain pct
    score = 0 # score for each session, an average of all available domains
    scores = []

    data = data.sort_values(by="end_time_min").reset_index() # sort data by time

    for idx, row in data.iterrows():
        updated_domain_pct[row["domain_id"]] = row["percentile"] # update domain pct to the latest one
        # find sum of all domain pct
        for k, v in updated_domain_pct.items():
            score += float(v)
        # take average of domain pct, add to list, reset score to 0
        score /= len(updated_domain_pct)
        scores.append(score)
        score = 0
    # set score to the score list
    data["score"] = scores
    return data

In [ ]:
## get performance metric by each patient for each session
final_df = df_pct.groupby("patient_id").apply(get_score).reset_index(drop=True)
final_df

## Question
* right now there are duplicate sessions since domain percentile was calulated separately, how do we feed this to the model?
* task changes visualization -> try to explain the fluctuations
* add overall timeline

* update only the domains that have been practiced
* keep other domains constant
* how much they switch domains

In [ ]:
import random
pid = random.choice(pd.unique(final_df["patient_id"]))
print(pid)

In [ ]:
pid = 57896
sns.scatterplot(data=final_df[final_df.patient_id == pid], x="end_time_min", y="score", hue="domain_id", palette="bright").set_title(pid)
plt.plot(final_df[final_df.patient_id == pid]["end_time_min"], final_df[final_df.patient_id == pid]["score"], 'k')

In [ ]:
sns.lineplot(data=final_df[final_df.patient_id == pid], x="end_time_min", y="progression_order", hue="domain_id", palette="bright").set_title(pid)

Second metric calculation:
* keep other domains constant 
* how do we start off each domain -> average of when people first start out or just average

In [ ]:
df_pct

First: average across all sessions

In [ ]:
df_pct.groupby("domain_id")["percentile"].mean()

In [ ]:
# take in individual dataframe and output dataframe with overall performance metric
def get_score_avg(data):
    # initialize score with average
    score = [0.5 for i in range(14)]
    scores = []
    

    data = data.sort_values(by="end_time_min") # sort data by time

    for idx, row in data.iterrows():
        score[row["domain_id"] - 1] = row["percentile"]
        # set score to the score list average
        scores.append(sum(score) / len(score))
    data["score"] = scores
    return data

In [ ]:
## get performance metric by each patient for each session
avg_final_df = df_pct.groupby("patient_id").apply(get_score_avg).reset_index(drop=True)
avg_final_df

Let's look at results

In [ ]:
import random
pid = random.choice(pd.unique(avg_final_df["patient_id"]))
print(pid)
sns.scatterplot(data=avg_final_df[avg_final_df.patient_id == pid], x="end_time_min", y="score", hue="domain_id", palette="bright").set_title(pid)
plt.plot(avg_final_df[avg_final_df.patient_id == pid]["end_time_min"], avg_final_df[avg_final_df.patient_id == pid]["score"])

Second average method: average starting point

In [ ]:
df_pct

In [ ]:
domain_avg = df_pct.sort_values(by="end_time_min")
domain_avg = domain_avg.drop_duplicates(subset=["patient_id", "domain_id"])

In [ ]:
domain_avg["domain_id"].unique()

In [ ]:
lst = list(domain_avg.groupby("domain_id")["percentile"].mean().reset_index()["percentile"])
lst

In [ ]:
# take in individual dataframe and output dataframe with overall performance metric
def get_score_avg2(data):
    # initialize score with average
    global lst
    score = lst
    scores = []
    
    data = data.sort_values(by="end_time_min") # sort data by time

    for idx, row in data.iterrows():
        score[row["domain_id"] - 1] = row["percentile"]
        # set score to the score list average
        scores.append(sum(score) / len(score))
    data["score"] = scores
    return data

In [ ]:
## get performance metric by each patient for each session
avg_final_df2 = df_pct.groupby("patient_id").apply(get_score_avg2).reset_index(drop=True)
avg_final_df2

In [ ]:
import random
pid = random.choice(pd.unique(avg_final_df2["patient_id"]))

print(pid)
sns.scatterplot(data=avg_final_df2[avg_final_df2.patient_id == pid], x="end_time_min", y="score", hue="domain_id", palette="bright").set_title(pid)
plt.plot(avg_final_df2[avg_final_df2.patient_id == pid]["end_time_min"], avg_final_df2[avg_final_df2.patient_id == pid]["score"])

Normalization

In [ ]:
df_pct

In [ ]:
sorted_df = df_pct.sort_values(by="end_time_min")
sorted_df

In [ ]:
# initial score
initial = lst

In [ ]:
column_names = ["domain %d score" % i for i in range(1, 15)]
column_names

In [ ]:
# create domain score columns, input a patient's session data
def create_domain_scores(data):
    global initial
    global column_names
    score = initial
    data = data.sort_values(by="end_time_min").reset_index() # sort data by time
    scores = np.zeros((len(data), 14))
    i = 0

    for idx, row in data.iterrows():
        score[row["domain_id"] - 1] = row["percentile"]
        # set score to the score list average
        scores[i] = score
        i += 1
    data = pd.concat([data, pd.DataFrame(scores, columns=column_names)], axis=1)
    return data

In [ ]:
scores_df = sorted_df.groupby("patient_id").apply(create_domain_scores).reset_index(drop=True)
scores_df

In [ ]:
# nomralize scores
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# scores only df
scores_df_minmax = MinMaxScaler().fit_transform(scores_df[column_names])

In [ ]:
minmax_df = scores_df
minmax_df[column_names] = scores_df_minmax

In [ ]:
minmax_df.columns

In [ ]:
minmax_df['score'] = minmax_df[column_names].mean(axis=1)
minmax_df

In [ ]:
avg_final_df2[avg_final_df2.patient_id == pid]

In [ ]:
minmax_df[minmax_df.patient_id == pid]

In [ ]:
import random
pid = random.choice(pd.unique(minmax_df["patient_id"]))
print(pid)
sns.scatterplot(data=minmax_df[minmax_df.patient_id == pid], x="end_time_min", y="score", hue="domain_id", palette="bright").set_title(pid)
plt.plot(minmax_df[minmax_df.patient_id == pid]["end_time_min"], minmax_df[minmax_df.patient_id == pid]["score"])

In [ ]:
temp = minmax_df[minmax_df.patient_id == pid]

In [ ]:
sns.lineplot(data=temp, x="start_time_min", y="score", hue="domain_id", palette="bright").set_title(pid)

In [ ]:
sns.lineplot(data=temp, x="start_time_min", y="progression_order", hue="domain_id", palette="bright").set_title(pid)

In [ ]:
temp = temp.sort_values(by="start_time_min")
sns.lineplot(data=temp[temp.task_type_id == 11], x="start_time_min", y="progression_order").set_title(pid)

In [ ]:
temp[["domain_id", "task_type_id", "task_level", "score", "progression_order", "domain 14 score", "end_time_min", "id"]]

Create day-to-day score -> combine sessions of the same domain -> average that then average across domains -> one score for each day instead of each session

Filter out fluctations

In [ ]:
# return false if there's fluctation given a person's data
def fluctate(data):
    data = data.sort_values(by="end_time_min").reset_index() # sort data by time
    d = dict()
    for idx, row in data.iterrows():
        if row["start_time"] not in d:
            d[row["start_time"]] = [row["domain_id"]]
        else:
            if row["domain_id"] in d[row["start_time"]]:
                return False
            else:
                d[row["start_time"]].append(row["domain_id"])
    return True

In [ ]:
filter_lst = minmax_df.groupby("patient_id").apply(fluctate).reset_index()

In [ ]:
filter_lst = filter_lst.rename(columns={0: "a"})
filter_lst

In [ ]:
filter_lst[filter_lst.a]["patient_id"]

In [ ]:
flutuate_data = minmax_df.loc[minmax_df["patient_id"].isin(filter_lst[filter_lst.a]["patient_id"])].reset_index()
flutuate_data

Filtering doesn't seem to work, work on incorporating accuracy

In [ ]:
df_pct

In [ ]:
def new_score(row):
    score = None
    p = [1.05, 1.0, 0.9, 0.8]
    if row.accuracy > .90:
        score = row.percentile * p[0]
    elif row.accuracy > .60:
        score = row.percentile * p[1]
    elif row.accuracy > .40:
        score = row.percentile * p[2]
    else:
        score = row.percentile * p[3]
    return score

In [ ]:
# 12 seconds
df_pscore = df_pct.copy(deep=True)
df_pscore["percentile"] = df_pscore.apply(new_score, axis=1)
df_pscore

In [ ]:
def combine(data):
    data = data.sort_values(by="end_time_min").reset_index() # sort data by time
    return data.groupby(["domain_id", "start_time"])["percentile"].mean().reset_index()

In [ ]:
# create domain score columns, input a patient's session data
def create_domain_scores2(data):
    global initial
    global column_names
    score = initial
    data = data.sort_values(by="end_time_min").reset_index() # sort data by time
    i = 0
    combined_scores = combine(data)
    scores = np.zeros((len(combined_scores), 14))
    combined_scores = combined_scores.sort_values(by="start_time").reset_index()
    for idx, row in combined_scores.iterrows():
        score[row["domain_id"] - 1] = row["percentile"]
        # set score to the score list average
        scores[i] = score
        i += 1
    data = data.drop_duplicates(subset=["domain_id", "start_time"]).reset_index()
    data = pd.concat([data, pd.DataFrame(scores, columns=column_names)], axis=1)
    return data

In [ ]:
df_pscore2 = df_pscore.groupby("patient_id").apply(create_domain_scores2).reset_index(drop=True)
df_pscore2

In [ ]:
column_n = column_names

In [ ]:
scores_df_minmax2 = MinMaxScaler().fit_transform(df_pscore2[column_n])

In [ ]:
minmax_df2 = df_pscore2
minmax_df2[column_n] = scores_df_minmax2

In [ ]:
minmax_df2['score'] = minmax_df2[column_n].mean(axis=1)
minmax_df2

In [ ]:
import random
pid = random.choice(pd.unique(minmax_df2["patient_id"]))
pid = 57896
print(pid)
sns.scatterplot(data=minmax_df2[minmax_df2.patient_id == pid], x="end_time_min", y="score", hue="domain_id", palette="bright").set_title(pid)
plt.plot(minmax_df2[minmax_df2.patient_id == pid]["end_time_min"], minmax_df2[minmax_df2.patient_id == pid]["score"], 'k')

In [ ]:
temp = df_pct[df_pct.patient_id == 30679]
temp = temp.sort_values(by="end_time_min").reset_index()
temp[temp.domain_id == 10][["accuracy", "progression_order", "percentile"]][temp.progression_order == 3]

In [ ]:
temp = create_domain_scores2(df_pscore[df_pscore.patient_id == 87299])
temp[temp.domain_id == 4][["accuracy", "progression_order", "domain 4 score", "start_time"]]